In [7]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from selenium import webdriver
from pathlib import Path
import time
import random
from itertools import repeat
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
# from selenium.webdriver.remote.remote_connection import LOGGER
# LOGGER.setLevel(logging.WARNING)
from concurrent.futures import ThreadPoolExecutor
import threading
import re
import csv
import itertools
import ast
import logging
logging.basicConfig(
    filename='scraping.log',
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logging.getLogger().setLevel(logging.INFO)

In [8]:
def filter_unscraped_iter(products_links_df,scraped_details_df):
    return list(products_links_df[~(products_links_df['ASIN'].isin(scraped_details_df['ASIN']))].itertuples(index=False))

In [9]:
products = pd.read_csv(r'aryaman/csv/categories_and_products/products.csv')

In [10]:
webdriver_path = "..\chromedriver.exe"
cService = webdriver.ChromeService(executable_path=webdriver_path)
# chrome_options = Options()
# chrome_options.add_experimental_option("detach",True)
driver = webdriver.Chrome(service=cService)

In [403]:
# with open('scraping_details.csv','w',newline='',encoding='utf-8') as file:
#     writer = csv.DictWriter(file,fieldnames=['ASIN', 'Images', 'title', 'productOverview', 'featureBullets', 'stars','ratings', 'listPrice', 'salePrice'])
#     writer.writeheader()

In [ ]:
# pd.DataFrame({'blacklisted_asins':[]}).to_csv('Blacklisted_ASINs.csv')

In [3]:
# pd.DataFrame({"ASIN":[],"star_rating":[],"ratings":[],"featureBullets":[],"Overview":[],"listPrice":[]}).to_csv('MissingDetailsChart.csv',index=False)

In [23]:
scraped_details_df = pd.read_csv('scraping_details.csv',encoding='utf-8')
products_links = products[['ASIN','Product Link']]
products_links.reset_index(inplace=True,drop=True)
blacklisted_asins = (pd.read_csv('Blacklisted_ASINs.csv'))['blacklisted_asins'].tolist()
missing_details_df = pd.read_csv('MissingDetailsChart.csv')
no_of_items_to_scrape = 1000
how_many_done_checkpoint = 0
with open('scraping_details.csv','a',newline='',encoding='utf-8') as file:
    writer = csv.DictWriter(file,fieldnames=['ASIN', 'Images', 'title', 'productOverview', 'featureBullets', 'stars','ratings', 'listPrice', 'salePrice'])
    iterator_to_loop = filter_unscraped_iter(products_links,scraped_details_df)
    for link in iterator_to_loop:
        if link.ASIN in blacklisted_asins:
            continue
        driver.get(link._1)
        time.sleep(1)
        missing_details_dict = {'ASIN':link.ASIN,'star_rating':False,'ratings':False,'featureBullets':False,'Overview':False}
        is_any_detail_missing = False
        try:
            img_src = driver.find_element(By.CSS_SELECTOR,"#imgTagWrapperId img").get_attribute("src")
            if len(img_src) == 0:
                logging.critical(f"no img src obtained : {link.ASIN} - {link._1}")
                raise ValueError
                
            title = driver.find_element(By.CSS_SELECTOR,"#productTitle").text
            if len(title) == 0:
                logging.critical(f"no title extracted : {link.ASIN} - {link._1}")
                raise ValueError
            try:
                star_rating = float(driver.find_element(By.ID,'acrPopover').get_attribute('title').split(" ")[0])
                if star_rating < 0 or star_rating > 5:
                    logging.critical(f"star rating messed up : {link.ASIN} - {link._1}")
                    raise ValueError
            except NoSuchElementException:
                logging.info(f"star rating element was not found : {link.ASIN} - {link._1}")
                is_any_detail_missing = True
                missing_details_dict['star_rating'] = True
                star_rating = None
            try:
                ratings = driver.find_element(By.ID,'acrCustomerReviewText').text
                ratings = re.sub(r'[^\d]', '',ratings)
                ratings = int(ratings)
                if ratings < 0 or (ratings > 0 and star_rating == 0):
                    logging.critical(f"some ratings messed up at index : {link.ASIN} - {link._1}")
                    raise ValueError
            except NoSuchElementException:
                logging.info(f"no of ratings element not found : {link.ASIN} - {link._1}")
                is_any_detail_missing = True
                missing_details_dict['ratings'] = True
                ratings = None
            overview = {}
            try:
                trs = driver.find_element(By.ID,'productOverview_feature_div').find_element(By.TAG_NAME,'table').find_elements(By.TAG_NAME,'tr')
                if len(trs) == 0:
                    logging.critical(f"no table rows of overview found : {link.ASIN} - {link._1}")
                    raise ValueError
                for tr in trs:
                    cells = tr.find_elements(By.TAG_NAME,'td')
                    if len(cells) == 2:
                        spans_in_first = cells[0].find_elements(By.TAG_NAME,'span')
                        spans_in_second = cells[1].find_elements(By.TAG_NAME,'span')
                        if len(spans_in_first) == len(spans_in_second) == 1:
                            overview[driver.execute_script("return arguments[0].textContent",spans_in_first[0])] = driver.execute_script("return arguments[0].textContent",spans_in_second[0])
            except NoSuchElementException:
                logging.info(f"overview feature not found : {link.ASIN} - {link._1}")
                is_any_detail_missing = True
                missing_details_dict['Overview'] = True
                overview = None
            try:
                bullets = driver.find_element(By.ID,'feature-bullets').find_element(By.TAG_NAME,'ul').find_elements(By.TAG_NAME,'li')
                if len(bullets) == 0:
                    logging.critical(f"no bullets of about this item found : {link.ASIN} - {link._1}")
                    raise ValueError
                bullets_text = list()
                for bullet in bullets:
                    span = bullet.find_element(By.TAG_NAME,'span')
                    bullets_text.append(driver.execute_script('return arguments[0].textContent',span))
            except NoSuchElementException:
                logging.info(f"bullets was not found : {link.ASIN} - {link._1}")
                is_any_detail_missing = True
                missing_details_dict['featureBullets'] = True
                bullets_text = None
            salePrice = float(re.sub(r'[^\d.]','',driver.find_element(By.ID,'corePriceDisplay_desktop_feature_div').find_element(By.CSS_SELECTOR,"span.priceToPay").find_element(By.CSS_SELECTOR,"span.a-price-whole").text))
            if salePrice < 0:
                logging.critical(f"sale price messed up : {link.ASIN} - {link._1}")
                raise ValueError
            try:
                basisPriceElem = driver.find_element(By.ID,'corePriceDisplay_desktop_feature_div').find_element(By.CSS_SELECTOR,"span.basisPrice").find_element(By.CSS_SELECTOR,"span.a-text-price").find_element(By.TAG_NAME,'span')
                basisPrice = driver.execute_script("return arguments[0].textContent",basisPriceElem)
                listPrice = float(re.sub(r'[^\d.]', '',basisPrice))
                if listPrice < salePrice:
                    logging.critical(f"listPrice smaller than sale price : {link.ASIN} - {link._1}")
                    raise ValueError
            except NoSuchElementException:
                logging.info(f'list price was not found : {link.ASIN} - {link._1}')
                is_any_detail_missing = True
                missing_details_dict['listPrice'] = True
                listPrice = None
            new_entry = {'ASIN':link.ASIN,'Images':[img_src],'title':title,'productOverview':overview,'featureBullets':bullets_text,'stars':star_rating,'ratings':ratings,'listPrice':listPrice,'salePrice':salePrice}
            writer.writerow(new_entry)
            file.flush()
            if is_any_detail_missing:
                missing_details_df = pd.concat([missing_details_df,pd.DataFrame([missing_details_dict])],ignore_index=True)
            no_of_items_to_scrape -= 1
            if no_of_items_to_scrape == 0:
                print("breaking normally")
                logging.info("-------------------------------------Next Lot Begins-------------------------------------------------------------")
                break
            how_many_done_checkpoint += 1
            if how_many_done_checkpoint == 100:
                missing_details_df.to_csv('MissingDetailsChart.csv',index=False)
                print(2000-no_of_items_to_scrape,"done")
                how_many_done_checkpoint = 0
        except ValueError as e:
            print("value Error occurred , breaking out ",e)
            break
        except NoSuchElementException:
            # print(f"no such element occurred , blacklisting {link.ASIN}")
            logging.warning(f'Blacklisted asin : {link.ASIN} - {link._1}')
            blacklisted_asins.append(link.ASIN)
            pd.DataFrame({'blacklisted_asins':blacklisted_asins}).to_csv('Blacklisted_ASINs.csv',index=False)
        except Exception as e:
            print("some exception occurred breaking out",e)
            logging.critical(f'some unexpected exception occurred : {link.ASIN} - {link._1}')
            break
missing_details_df.to_csv('MissingDetailsChart.csv',index=False)

In [329]:
df['Images'] = df['Images'].apply(lambda x :ast.literal_eval(x))
df['productOverview'] = df['productOverview'].apply(lambda x :ast.literal_eval(x))
df['featureBullets'] = df['featureBullets'].apply(lambda x :ast.literal_eval(x))

In [477]:
products

,Main Category,Tag,ASIN,Product Link
0,Baby Products,Diapers,B075GHP2LX,https://www.amazon.in/dp/B075GHP2LX
1,Baby Products,Diapers,B08X3KG1T9,https://www.amazon.in/dp/B08X3KG1T9
2,Baby Products,Diapers,B0D9GX6F1W,https://www.amazon.in/dp/B0D9GX6F1W
3,Baby Products,Diapers,B07DP24NSR,https://www.amazon.in/dp/B07DP24NSR
4,Baby Products,Diapers,B08X3LXQT7,https://www.amazon.in/dp/B08X3LXQT7
...,...,...,...,...
7087,Sports & Outdoors,Water Bottles & Shakers,B0BF5YFP1K,https://www.amazon.in/dp/B0BF5YFP1K
7088,Sports & Outdoors,Water Bottles & Shakers,B08Y6Z1TCH,https://www.amazon.in/dp/B08Y6Z1TCH
7089,Sports & Outdoors,Water Bottles & Shakers,B0CKQ2PV14,https://www.amazon.in/dp/B0CKQ2PV14
7090,Sports & Outdoors,Water Bottles & Shakers,B0CQSTXRHN,https://www.amazon.in/dp/B0CQSTXRHN


In [478]:
blacklisted_asins

['B0DGTRFQTL', 'B0DGTNQ8BD', 'B0DGTQWPTQ', 'B0DC3R87QV', 'B0DJSKND2G']

: 